In [1]:
import sys
try:
    import sntd
except:
    sys.path.append('../')
    import sntd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
myMISN = sntd.createMultiplyImagedSN(sourcename='salt2-extended', snType='Ia', redshift=1.83,z_lens=.53, bands=['F110W','F125W','F140W','F160W'],
              zp=[26.9,26.8,26.4,26.2], cadence=5., epochs=35.,time_delays=[20., 70.], magnifications=[4,6],scatter=True,
              objectName='My Type Ia SN',telescopename='HST',numImages=2,av_host=False)

In [3]:
#normal parallel fit with some options set
fitCurves=sntd.fit_data(myMISN,snType='Ia', models='salt2-extended',bands=['F814W','F110W','F125W','F140W'],
            params=['x0','t0','x1','c'],constants={'z':1.83},refImage='image_1',cut_time=[-50,30],
            bounds={'t0':(-20,20),'x1':(-3,3),'c':(-1,1),'td':(-20,20)},identify_micro=True,
            fitOrder=['image_2','image_1'],fit_prior=None,trial_fit=True,minsnr=3,
            method='parallel',microlensing=None,modelcov=False,npoints=100,clip_data=True,maxiter=None)
fitCurves.plot_object(showFit=True)
for im in fitCurves.images.keys():
    fitCurves.plot_fit(par_image=im)
print(fitCurves.parallel.time_delays)

F125W-F110W -23.15015113188333 nan
F140W-F125W -24.826350357027323 nan
F140W-F110W -23.15015113188333 nan
F125W-F110W -23.651319747354655 nan
[] ['F140W' 'F110W']


SystemExit: 